##Install Dependencies


In [1]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.3 MB/s eta 0:00:00


#Importing 

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests #scrap the webpage
from bs4 import BeautifulSoup #extract data from the DOM
import re #create a regex function to extract the comments we want

In [4]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [5]:
tokens=tokenizer.encode('It was not that bad.', return_tensors='pt')

In [6]:
tokens[0]


tensor([  101, 10197, 10140, 10497, 10203, 12428,   119,   102])

In [7]:
result=model(tokens)

In [8]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-1.8060, -0.2894,  1.7198,  1.0222, -0.5435]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [9]:
result.logits


tensor([[-1.8060, -0.2894,  1.7198,  1.0222, -0.5435]],
       grad_fn=<AddmmBackward0>)

In [10]:
int(torch.argmax(result.logits))+1

3

#Collecting reviews from a webpage

In [11]:
r=requests.get('https://www.yelp.com/biz/nara-restaurant-and-sake-bar-san-francisco')
soup=BeautifulSoup(r.text, 'html.parser')
regex=re.compile('.*comment.*')
results=soup.find_all('p', {'class':regex})
reviews=[result.text for result in results]

#Loading into a dataframe

In [14]:
import pandas as pd
import numpy as np
df=pd.DataFrame(np.array(reviews), columns=['review'])

In [15]:
df.tail()

,review
5,"Nara has a certain ""je ne sais quoi"" to the me..."
6,Have been here on dates and took my family her...
7,Really nice dinner spot-- perfect for a date n...
8,My best friend and I came here on a weekday po...
9,"This is my second time here, but first time le..."


In [17]:
df['review'].iloc[0]

'Food was great, service was even better. High quality seafood. A must try. Has great ambience as well.'

In [18]:
def sentiment_score(review):
  tokens=tokenizer.encode(review, return_tensors='pt')
  result=model(tokens)
  return int(torch.argmax(result.logits))+1

In [19]:
sentiment_score(df['review'].iloc[0])

5

In [20]:
df['sentiment']=df['review'].apply(lambda x: sentiment_score(x[:512]))

In [21]:
df['sentiment']

0    5
1    4
2    5
3    4
4    5
5    3
6    4
7    4
8    5
9    4
Name: sentiment, dtype: int64

In [22]:
df


,review,sentiment
0,"Food was great, service was even better. High ...",5
1,they're sooo nice and accommodated my gluten a...,4
2,Ordered: Hamachi Truffle Wagyu TartareNigiri O...,5
3,I wish I lived closer so I could be a regular ...,4
4,This restaurant was such a wonderful experienc...,5
5,"Nara has a certain ""je ne sais quoi"" to the me...",3
6,Have been here on dates and took my family her...,4
7,Really nice dinner spot-- perfect for a date n...,4
8,My best friend and I came here on a weekday po...,5
9,"This is my second time here, but first time le...",4


In [23]:
df['review'].iloc[5]

'Nara has a certain "je ne sais quoi" to the menu, which I figured out was that the menu is Japanese fares made by Koreans. You\'ll find non-traditional rolls like the loco Spider-Man roll and special K which are not anything you\'d find in a typical sushi restaurant. The special K roll was kind of fishy, so the seared butter fish on top was not my favorite. Next time, I would just skip rolls all together and go for omakase or a La carte nigiri.The most stars go to the nigiri, which are pretty average-sized slices of fish. I did enjoy the uni, which were very sweet and fresh from Baja. The unagi was paired with grated ginger, which was a nice, refreshing touch. The golden eye snapper (kinmedai) was firm and tender. The service was slower as it was pretty packed, but the waitress was apologetic and accommodating. At the price point, I would prefer to go to other restaurants - so Nara is ultimately 3.5 stars being round up to 4 for the experience.'